# Data Analysis with Python and Pandas Tutorial
# Part 2 - Selecting, Filtering and Aggregating data

## Tutorial Objectives

In this tutorial, you will learn:

  * How to explore of a large dataset using shape, columns, detailed info, identify categorical data, and exploring numerical columns distribution
  * How to do basic clean up including renaming columns, converting relevant columns to categorical data, and dropping irrelevant columns
  * Locate specific rows and cells using iloc and loc
  * Find the largest and smallest values in a row using nlargest and nsmallest
  * Select specific columns
  * Filter rows using various criteria
  * Aggregating data using mena. min, max, and sum to analyse data
  * Plot time-based series

## Loading the dataset

In [ ]:
# import the Pandas library
import pandas as pd

In [ ]:
# read the dataset avocado.csv, available at https://www.kaggle.com/neuromusic/avocado-prices/download
df = pd.read_csv('avocado.csv', index_col=0)
df

## Exploring the dataset

In [ ]:
# output the dataframe's shape
df.shape

In [ ]:
# output the first few rows
df.head()

In [ ]:
# output the list of columns
df.columns

In [ ]:
# or, output detailed dataframe info
df.info()

In [ ]:
# rename some columns to not have spaces
col_mapping = {
    'Total Volume': 'TotalVolume',
    'Total Bags': 'TotalBags',
    'Small Bags': 'SmallBags',
    'Large Bags': 'LargeBags',
    'XLarge Bags': 'XLargeBags'
}
df.rename(columns=col_mapping, inplace=True)

In [ ]:
# drop some columns
drop_cols = ['4046', '4225', '4770', 'SmallBags', 'LargeBags', 'XLargeBags']

df.drop(columns=drop_cols, inplace=True)

In [ ]:
# convert column "Date" to datetime64
df.Date = df.Date.astype('datetime64')

In [ ]:
# output unique values in Series "type"
df.type.value_counts()

In [ ]:
# convert type to categorical
df.type = df.type.astype('category')

In [ ]:
# output unique values in Series "year"
df.year.value_counts()

In [ ]:
# convert year to categorical
df.year = df.year.astype('category')

In [ ]:
# output the (number of) regions
len(df.region.unique())

In [ ]:
# output unique values in Series "region" (notice "TotalUS")
df.region.unique()

In [ ]:
# drop all rows with region TotalUS
# this should be done *before* converting the column to category
df = df[df.region != 'TotalUS']

In [ ]:
# convert some columns to categorical Series
df.region = df.region.astype('category')

In [ ]:
# review the cleaned up columns (notice the corrected column names, types, and lowered memory usage)
df.info()

In [ ]:
# review the cleaned up dataset
df.head()

In [ ]:
# output basic statistics (numeric series)
df.describe()

In [ ]:
# plot a histogram of the average sale price
df.AveragePrice.plot(kind='hist', figsize=(8,4), title='Avocado Prices (USD)')

## Selecting Data

In [ ]:
# output the head again
df.head()

In [ ]:
# locate a specific row by index (e.g. row index 3)
df.iloc[3]

In [ ]:
# locate a specific value by index (e.g. row 3, column TotalVolume which is column index 2)
df.iloc[3, 2]

In [ ]:
# locate a specific value by name (row 3, column TotalVolume)
df.iloc[3]['TotalVolume']

In [ ]:
# grab a specific column, e.g. Average price
# notice the notation - we could also have written df.AveragePrice
df['AveragePrice'].head()

In [ ]:
# grab multiple columns, e.g. Date and AveragePrice
df[['Date', 'AveragePrice']].head()

In [ ]:
# Find the rows with the smallest sales volume, all columns
df.nsmallest(5, columns='TotalVolume')

In [ ]:
# Find the rows with the largest sales volume, single column
# notice df.TotalVolume could have been written as df['TotalVolume']
df.TotalVolume.nlargest()

## Filtering Data

In [ ]:
# Get rows with a TotalVolume greater than 10M
df[(df.TotalVolume > 10000000)].head()

In [ ]:
# Get rows for California
df[(df.region == 'California')].head()

In [ ]:
# Get rows for California with TotalVolume greater than 10M
df[(df.region == 'California') & (df.TotalVolume > 10000000)].head()

In [ ]:
# get rows for Portland and Orlando (output both head and tail)
df[df.region.isin(['Portland', 'Orlando'])].head()

In [ ]:
# get rows with an AveragePrice < 0.5
# select columns Date, AveragePrice, TotalVolume, region, type
cols = ['Date', 'AveragePrice', 'TotalVolume', 'region', 'type']

df.loc[df.AveragePrice < 0.5, cols].head()


## Grouping and Aggregating Data

### Research questions

Answer the following questions, supported by data:
  * What is the price difference between the different avocado types?
  * What regions have the highest/lowest avocado prices?
  * How does the price compare in regions with high vs low volume?
  * How have avocado prices developed over time?
  * What month is avocado peak season?

In [ ]:
# group the dataset by year, and output the first row of each group
df.groupby(by='year').first()

In [ ]:
# get the data of a specific group
df.groupby('region').get_group('Detroit').head()

In [ ]:
# get average avocado prices, by type
df.groupby(by='type').mean()

In [ ]:
# get average avocado prices, by region and type
df.groupby(by=['region', 'type']).mean().head(10)

In [ ]:
# get averages for all numeric columns, by region
df.groupby(by='region').mean().head()

In [ ]:
# get the average avocado price by region, lowest 5 prices only
df.groupby(by='region').mean().nsmallest(n=5, columns='AveragePrice')

In [ ]:
# get the average avocado price by region, highest 5 prices only
df.groupby(by='region').mean().nlargest(n=5, columns='AveragePrice')

In [ ]:
# for all regions, get the min price, mean price, max price, and the sum of total volume and bags
agg_funcs = {
    'AveragePrice': ['min', 'mean', 'max'],
    'TotalVolume': 'sum',
    'TotalBags': 'sum'
}
df.groupby('region').agg(agg_funcs).head()

In [ ]:
# get similar aggregates as above by region. But this time, get the 5 largest TotalVolume
df.groupby('region').agg(agg_funcs).nlargest(5, columns=('TotalVolume', 'sum'))

In [ ]:
# get similar aggregates as above by region. But this time, get the 5 smallest TotalVolume
df.groupby('region').agg(agg_funcs).nsmallest(5, columns=('TotalVolume', 'sum'))

In [ ]:
# group the dataframe by Date, calculating the mean for each column
# plot the dataframe. AveragePrice on the secondary y axis. add title, add grid, change figsize
df.groupby(by=['Date']).mean().plot(secondary_y=['AveragePrice'],
                                    title='Avocado Prices (2015-2018)',
                                    figsize=(12,5),
                                    grid=True)

## Exercise

Go ahead and load the tips dataset, do basic cleanups of columns, and then ask research questions such as:
  * The tips are what percentage of total bills? (new column)
  * What is the bill per person? Tips per person (new columns: divide total bill / tips by size)
  * At what time are tips higher?
  * What days of the week have higher spending in restaurants?
  * Do female waitresses get higher tips?

The dataset is available at:
https://raw.github.com/pydata/pandas/master/pandas/tests/data/tips.csv

To answer the questions, you need to aggregate data and output relevant tables.

Discuss your solutions with the person next to you!